In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt, gc
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def denoise_filter(x):
    # Sample rate and desired cutoff frequencies (in Hz).
    fs = 200.0
    lowcut = 1.0
    highcut = 25.0
    
    # Filter a noisy signal.
    T = 50
    nsamples = T * fs
    t = np.arange(0, nsamples) / fs
    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=6)
    y = (y + np.roll(y,-1)+ np.roll(y,-2)+ np.roll(y,-3))/4
    y = y[0:-1:4]
    
    return y

In [ ]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].apply(lambda x: x.mode().iloc[0]).reset_index()
tmp2 = df.groupby(['eeg_id','expert_consensus'])[['eeg_sub_id']].agg(min).reset_index()
tmp = pd.merge(tmp,tmp2,on=['eeg_id','expert_consensus'],how='left')
train['target'] = tmp['expert_consensus'].values
train['eeg_sub_id'] = tmp['eeg_sub_id'].values

train = pd.merge(train,df[['eeg_id','eeg_sub_id','eeg_label_offset_seconds']],on=['eeg_id','eeg_sub_id'],how='left')
train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

In [ ]:
NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'

In [ ]:
def quantize_data(data, classes):
    data = data-data.mean()
    data = data/(abs(data).max())
    mu_x = mu_law_encoding(data, classes)
    return mu_x#quantized

def mu_law_encoding(data, mu):
    mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
    return mu_x


In [ ]:
all_eegs1 = {}
all_eegs2 = {}
for idx in tqdm(range(len(train))):
    row = train.iloc[idx]
    eeg_id = row['eeg_id']
    eeg_sub_id = row['eeg_sub_id']
    parq_path = f'{PATH}{eeg_id}.parquet'
    eeg = pd.read_parquet(parq_path)
    start = int(row['eeg_label_offset_seconds']*200)
    eeg = eeg.iloc[start:start+10000]
    eeg = eeg.fillna(0)
    eeg = eeg.drop(['Fz', 'Cz','Pz','EKG'], axis=1)
#     eeg = eeg-eeg.min()
#     eeg = eeg/(eeg.abs().max())
#         print(idx,eeg_id,start,eeg.shape)
    signals1 = []
    signals2 = []
    for k in range(4):
            COLS = FEATS[k]

            # COMPUTE PAIR DIFFERENCES AND AVERAGE
#             x = eeg[COLS[0]].values - eeg[COLS[1]].values
            for j in range(4):
                x = eeg[COLS[j]].values - eeg[COLS[j+1]].values
                x = denoise_filter(x)
                signals1.append(x)
                x = quantize_data(x,1)
                signals2.append(x)
    signals1 = np.array(signals1,dtype='float32')
    signals2 = np.array(signals2,dtype='float32')
    all_eegs1[f'{eeg_id}_{eeg_sub_id}'] = signals1
    all_eegs2[f'{eeg_id}_{eeg_sub_id}'] = signals2
    
np.save('16_waves_eeg_specs_partial_train',all_eegs1)
np.save('16_waves_eeg_specs_partial_train_quantize',all_eegs2)

## Dataloader